In [6]:
from __future__ import absolute_import, division,print_function,unicode_literals
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
'''We'll use the IMDB dataset that contains the text of 50,000 movie reviews from the Internet Movie Database. 
These are split into 25,000 reviews for training and 25,000 reviews for testing. 
The training and testing sets are balanced, 
meaning they contain an equal number of positive and negative reviews.'''
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

GPU is NOT AVAILABLE


In [14]:
# split the training set inot 60& and 40%
#for training 15,000 examples, 10,000 for validation and 25,000 for testing

train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])
(train_data,validation_data),test_data=tfds.load(
    name="imdb_reviews",
    split=(train_validation_split,tfds.Split.TEST),
    as_supervised=True)

W0803 14:45:34.851153 20612 deprecation.py:323] From c:\users\nitin mali\appdata\local\programs\python\python37\lib\site-packages\tensorflow_datasets\core\file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to C:\Users\Nitin Mali\tensorflow_datasets\imdb_reviews\plain_text\0.1.0. Subsequent calls will reuse this data.


In [18]:
#Explore the data 
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: id=224, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [44]:
train_labels_batch

<tf.Tensor: id=225, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int64)>

In [ ]:
#Build the model 
'''The neural network is created by stacking layers—this requires three main architectural decisions:

How to represent the text?
How many layers to use in the model?
How many hidden units to use for each layer?'''

'''One way to represent the text is to convert sentences into embeddings vectors. 
We can use a pre-trained text embedding as the first layer, which will have three advantages: * 
we don't have to worry about text preprocessing, * 
we can benefit from transfer learning, * the embedding has a fixed size, so it's simpler to process.'''

In [ ]:
'''For this example we will use a pre-trained text embedding model from TensorFlow Hub called google/tf2-preview/gnews-swivel-20dim/1.

There are three other pre-trained models to test for the sake of this tutorial:

google/tf2-preview/gnews-swivel-20dim-with-oov/1 - same as google/tf2-preview/gnews-swivel-20dim/1, but with 2.5% vocabulary converted to OOV buckets. This can help if vocabulary of the task and vocabulary of the model don't fully overlap.
google/tf2-preview/nnlm-en-dim50/1 - A much larger model with ~1M vocabulary size and 50 dimensions.
google/tf2-preview/nnlm-en-dim128/1 - Even larger model with ~1M vocabulary size and 128 dimensions.
Let's first create a Keras layer that uses a TensorFlow Hub model to embed the sentences, and try it out on a couple of input examples. Note that no matter the length of the input text, the output shape of the embeddings is: (num_examples, embedding_dimension).'''

In [45]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=565, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [47]:
#example for sentence converted to 50 dimensions output vector
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1")
embeddings = embed(["cat is on the mat", "dog is in the fog"])

In [56]:
#S let's now build the full model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_12 (Dense)             (None, 16)                336       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [ ]:
'''The layers are stacked sequentially to build the classifier:

The first layer is a TensorFlow Hub layer. This layer uses a pre-trained Saved Model to map a sentence into its embedding vector. 
The pre-trained text embedding model that we are using (google/tf2-preview/gnews-swivel-20dim/1) 
splits the sentence into tokens, embeds each token and then combines the embedding.
The resulting dimensions are: (num_examples, embedding_dimension).
This fixed-length output vector is piped through a fully-connected (Dense) layer with 16 hidden units.
The last layer is densely connected with a single output node. Using the sigmoid activation function, 
this value is a float between 0 and 1, representing a probability, or confidence level.'''

In [ ]:
'''Loss function and optimizer
A model needs a loss function and an optimizer for training. Since this is a binary classification problem and the model outputs a probability (a single-unit layer with a sigmoid activation), we'll use the binary_crossentropy loss function.

This isn't the only choice for a loss function, you could, for instance, choose mean_squared_error. But, generally, binary_crossentropy is better for dealing with probabilities—it measures the "distance" between probability distributions, or in our case, between the ground-truth distribution and the predictions.

Later, when we are exploring regression problems (say, to predict the price of a house), we will see how to use another loss function called mean squared error.

Now, configure the model to use an optimizer and a loss function:'''

In [57]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
'''Train the model for 20 epochs in mini-batches of 512 samples. 
This is 20 iterations over all samples in the x_train and y_train tensors. 
While training, monitor the model's loss and accuracy on the 10,000 samples from the validation set:'''

In [59]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================].6343 - accuracy: 0.63 - 2s 1s/step - loss: 0.6480 - accuracy: 0.62 - 3s 866ms/step - loss: 0.6548 - accuracy: 0.615 - 3s 716ms/step - loss: 0.6461 - accuracy: 0.628 - 3s 621ms/step - loss: 0.6425 - accuracy: 0.631 - 3s 571ms/step - loss: 0.6432 - accuracy: 0.634 - 4s 536ms/step - loss: 0.6457 - accuracy: 0.630 - 4s 507ms/step - loss: 0.6443 - accuracy: 0.633 - 4s 482ms/step - loss: 0.6442 - accuracy: 0.635 - 5s 457ms/step - loss: 0.6426 - accuracy: 0.638 - 5s 435ms/step - loss: 0.6415 - accuracy: 0.639 - 5s 416ms/step - loss: 0.6389 - accuracy: 0.640 - 5s 396ms/step - loss: 0.6380 - accuracy: 0.642 - 5s 382ms/step - loss: 0.6370 - accuracy: 0.642 - 6s 368ms/step - loss: 0.6385 - accuracy: 0.641 - 6s 355ms/step - loss: 0.6397 - accuracy: 0.641 - 6s 343ms/step - loss: 0.6394 - accuracy: 0.639 - 6s 333ms/step - loss: 0.6382 - accuracy: 0.641 - 6s 323ms/step - loss: 0.6377 - accuracy: 0.641 - 6s 316ms/step - loss: 0.6358 - accuracy: 0.644 -

In [60]:
'''Evaluate the model
And let's see how the model performs. Two values will be returned. Loss (a number which represents our error, 
lower values are better), and accuracy.'''

results = model.evaluate(test_data.batch(512), verbose=0)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.417
accuracy: 0.816
